### `Re-quantification`

The re-quantification consists of 7 steps

![Re-quantification.png](images/Re-quantification.png)

In [ ]:
import os
import shutil
import glob
import numpy as np
import pandas as pd
from pyopenms import *
import plotly.express as px

#### `1) Split consensus`

 Split the ConsensusMap into features that have no missing values, and features that have at least one missing value; requantify only the missing values. 

In [ ]:
interim= os.path.join("results", "interim")
path = os.path.join(interim, "Requantification")
if not os.path.exists(path): # if it doesn't exist
    os.mkdir(path) # create a path directory
    
# split ConsensusMap
consensus_map = ConsensusMap()
ConsensusXMLFile().load(os.path.join(interim, "Preprocessing", "consensus" + ".consensusXML"), consensus_map)

headers = consensus_map.getColumnHeaders()

complete = ConsensusMap(consensus_map)
complete.clear(False)
missing = ConsensusMap(consensus_map)
missing.clear(False)

for cf in consensus_map:
    if len(cf.getFeatureList()) < len(headers): #missing values
        missing.push_back(cf)
    else:
        complete.push_back(cf) #no missing values

ConsensusXMLFile().store(os.path.join(path, "consensus_complete" + ".consensusXML"), complete)
ConsensusXMLFile().store(os.path.join(path, "consensus_missing" + ".consensusXML"), missing)

# get intensities as a DataFrame
result = missing.get_df()
result= result.reset_index()
result= result.drop(columns= ["sequence"])
# store as tsv file
result.to_csv(os.path.join(path, "FeatureMatrixNaN.tsv"), sep = "\t", index = False)
result

# reconstruct complete FeatureMaps
consensus_map = ConsensusMap()
ConsensusXMLFile().load(os.path.join(path, "consensus_complete" + ".consensusXML"), consensus_map)

featurexml_files= glob.glob(os.path.join(interim, "Preprocessing", 'MapAligned_*.featureXML'))
feature_maps = []
for featurexml_file in featurexml_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

to_keep_ids = [item for sublist in [[feature.getUniqueId() for feature in cf.getFeatureList()] for cf in consensus_map] for item in sublist]

for fm in feature_maps:
    fm_filterd = FeatureMap(fm)
    fm_filterd.clear(False)
    for f in fm:
        if f.getUniqueId() in to_keep_ids:
            fm_filterd.push_back(f)
    FeatureXMLFile().store(os.path.join(path, "Complete_" + os.path.basename(fm_filterd.getMetaValue("spectra_data")[0].decode())[7:-4] + "featureXML"), fm_filterd)

#### `2) FeatureFinderMetaboIdent: re-quantify`
This algorithm detects and extracts MS1 data that match the feature list in the metabolite identification table.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/UTILS_FeatureFinderMetaboIdent.html

##### `2) (i) Create a library of metabolites`

In [ ]:
#Import the feature matrix tsv table and keep only the columns: RT, mz and charge
DF_features = pd.read_csv(os.path.join(path, "FeatureMatrixNaN.tsv"), sep="\t")
DF_features = DF_features[["RT","mz", "charge"]]

DF_features= DF_features.rename(columns={ "charge":"Charge", "mz": "Mass", "RT": "RetentionTime"})

DF_features["Charge"]= DF_features["Charge"].astype(str)
DF_features["Mass"]= DF_features["Mass"].astype(float)

#For positive ionisation: comment this for negative ESI

for ind in DF_features.index:
    if DF_features["Charge"][ind] == "0":
        DF_features.loc[ind, "Mass"]= DF_features.loc[ind,"Mass"]- 1.007825
        DF_features.loc[ind, "Charge"]= "+1"
    if DF_features["Charge"][ind] == "1":
        DF_features.loc[ind, "Mass"]= DF_features.loc[ind,"Mass"]- 1.007825
        DF_features.loc[ind, "Charge"]= "+" + DF_features.loc[ind,"Charge"]
    if DF_features["Charge"][ind] == "2":
        DF_features.loc[ind, "Mass"]= (DF_features.loc[ind,"Mass"]*2)- 2.015650
        DF_features.loc[ind, "Charge"]= "+" + DF_features.loc[ind,"Charge"]
    if DF_features["Charge"][ind] == "3":
        DF_features.loc[ind, "Mass"]= (DF_features.loc[ind,"Mass"]*3)- 3.023475
        DF_features.loc[ind, "Charge"]= "+" + DF_features.loc[ind,"Charge"]

DF_features["CompoundName"] = np.arange(len(DF_features))
DF_features["CompoundName"] = "feature_" + DF_features["CompoundName"].astype(str)
DF_features["SumFormula"] = " "
DF_features["RetentionTimeRange"]= "0"
DF_features["IsoDistribution"]= "0"
DF_features= DF_features[["CompoundName","SumFormula", "Mass","Charge","RetentionTime","RetentionTimeRange", "IsoDistribution"]]
DF_features.to_csv(os.path.join(path, "MetaboliteIdentification.tsv"), sep="\t", index= None)
DF_features

In [ ]:
# For negative ionisation: uncomment this for negative ESI
    # for ind in DF_features.index:
    #     if DF_features["Charge"][ind] == "0":
    #         DF_features.loc[ind, "Mass"]= DF_features.loc[ind,"Mass"]+ 1.007825
    #     if DF_features["Charge"][ind] == "1":
    #         DF_features.loc[ind, "Mass"]= DF_features.loc[ind,"Mass"]+ 1.007825
    #     if DF_features["Charge"][ind] == "2":
    #         DF_features.loc[ind, "Mass"]= (DF_features.loc[ind,"Mass"]*2)+ 2.015650
    #     if DF_features["Charge"][ind] == "3":
    #         DF_features.loc[ind, "Mass"]= (DF_features.loc[ind,"Mass"]*3)+ 3.023475
    # DF_features["Charge"]= DF_features["Charge"].astype(str)
    # for ind in DF_features.index:
    #     if DF_features["Charge"][ind] == "0":
    #         DF_features.loc[ind, "Charge"]= "-1"
    #     if DF_features["Charge"][ind] == "1":
    #         DF_features.loc[ind, "Charge"]= "-" + DF_features.loc[ind,"Charge"]
    #     if DF_features["Charge"][ind] == "2":
    #         DF_features.loc[ind, "Charge"]= "-" + DF_features.loc[ind,"Charge"]
    #     if DF_features["Charge"][ind] == "3":
    #         DF_features.loc[ind, "Charge"]= "-" + DF_features.loc[ind,"Charge"]

In [ ]:
import csv
# read tsv file and create list of FeatureFinderMetaboIdentCompound
def metaboTableFromFile(path_to_library_file):
    metaboTable = []
    with open(path_to_library_file, "r") as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        next(tsv_reader) # skip header
        for row in tsv_reader:
            metaboTable.append(FeatureFinderMetaboIdentCompound(
                row[0], # name
                row[1], # sum formula
                float(row[2]), # mass
                [int(charge) for charge in row[3].split(",")], # charges
                [float(rt) for rt in row[4].split(",")], # RTs
                [float(rt_range) for rt_range in row[5].split(",")], # RT ranges
                [float(iso_distrib) for iso_distrib in row[6].split(",")] # isotope distributions
            ))
    return metaboTable

##### `2) (ii) Requantify mzML files`

In [ ]:
input_mzml_files=sorted(glob.glob(os.path.join(interim, 'Preprocessing', 'MapAligned_*.mzML')))

# load ms data from mzML file into MSExperiment
for mzml_file in input_mzml_files:
    spectra = MSExperiment()
    MzMLFile().load(mzml_file, spectra)

    # create FeatureFinderAlgorithmMetaboIdent and assign ms data
    ffmid = FeatureFinderAlgorithmMetaboIdent()
    ffmid.setMSData(spectra)

    params = ffmid.getParameters()
    params[b"extract:mz_window"] = 10.0 
    params[b"detect:peak_width"] = 60.0  #adjust for wide peaks
    ffmid.setParameters(params)
    # FeatureMap to store results
    fm = FeatureMap()

    # run the FeatureFinderMetaboIdent with the metabo_table and store results in fm
    metabo_table = metaboTableFromFile(os.path.join(path, "MetaboliteIdentification.tsv"))
    ffmid.run(metabo_table, fm, String(mzml_file))
    # set number of mass traces (for SIRIUS)
    fm_include_mass_traces = FeatureMap(fm)
    fm_include_mass_traces.clear(False)
    for feature in fm:
        feature.setMetaValue("num_of_masstraces", params[b"extract:n_isotopes"])
        fm_include_mass_traces.push_back(feature)
    fm = fm_include_mass_traces
    
    # save FeatureMap to file
    ff_file = os.path.join(path, "FFMID_" + os.path.basename(mzml_file)[11:-5] +".featureXML")
    FeatureXMLFile().store(ff_file, fm)

#### `4) Merge Feature maps`

Merge complete FeatureMaps from FFM with requantified FeatureMaps from FFMID

In [ ]:
for complete_map in sorted(glob.glob(os.path.join(path, "Complete_*.featureXML"))):
    for requant_map in sorted(glob.glob(os.path.join(path, "FFMID_*.featureXML"))):
        if os.path.basename(complete_map)[9:] == os.path.basename(requant_map)[6:]:
            fm_ffm = FeatureMap()
            FeatureXMLFile().load(complete_map, fm_ffm)
            fm_ffmid = FeatureMap()
            FeatureXMLFile().load(requant_map, fm_ffmid)
            for f in fm_ffmid:
                fm_ffm.push_back(f)
            fm_ffm.setUniqueIds()
            FeatureXMLFile().store(os.path.join(path, "Merged_" + os.path.basename(complete_map)[9:]), fm_ffm)

#### `5) Metabolite adduct decharger` 

For each peak, this algorithm reconstructs neutral masses by enumerating all possible adducts with matching charge. Here, we do not save the file with neutral masses, but only the feature files that contain adduct annotations. You can add the list of adduct for the algorithm to parse through. SIRIUS, an algorithm that is later used, is only able to compute singly charged adducts so charges higher than 1 are filtered out. Use adduct list: [b"H-1:-:1", b"H-2O-1:0:0.05", b"CH2O2:0:0.5"] for negative mode.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/UTILS_MetaboliteAdductDecharger.html

In [ ]:
input_feature_files= sorted(glob.glob(os.path.join(path, "Merged_*.featureXML")))

for filename in input_feature_files:
        feature_map = FeatureMap()    
        FeatureXMLFile().load(filename, feature_map)
        mfd = MetaboliteFeatureDeconvolution()
        mdf_par = mfd.getDefaults()
        mdf_par.setValue("potential_adducts", [b"H:+:0.4",b"Na:+:0.2",b"NH4:+:0.2", b"H-1O-1:+:0.1", b"H-3O-2:+:0.1"])
        mdf_par.setValue("charge_min", 1, "Minimal possible charge")
        mdf_par.setValue("charge_max", 1, "Maximal possible charge")
        mdf_par.setValue("charge_span_max", 1)
        mdf_par.setValue("max_neutrals", 1)
        mfd.setParameters(mdf_par)
        feature_map_MFD = FeatureMap()
        cons_map0 = ConsensusMap()
        cons_map1 = ConsensusMap()
        mfd.compute(feature_map, feature_map_MFD, cons_map0, cons_map1)
        featurefile = os.path.join(path, "MFD_" + os.path.basename(filename)[7:])
        FeatureXMLFile().store(featurefile, feature_map_MFD)

Display in a dataframe

In [ ]:
input_feature_files = glob.glob(os.path.join(path, "MFD_*.featureXML"))

path = os.path.join("results", "features")
if not os.path.exists(path): # if it doesn't exist
    os.mkdir(path) # create a path directory

for filename in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(filename, fmap)
    DF= fmap.get_df(export_peptide_identifications=False)
    for f in fmap:
            if f.metaValueExists("dc_charge_adducts"):
                DF["adduct"] = [f.getMetaValue("dc_charge_adducts") for f in fmap]
    feature_csv= os.path.join(path, "features_" + os.path.basename(filename)[4:-10] +"csv")
    DF.to_csv(feature_csv)
print("example:", os.path.basename(filename))
display(DF)

#### `6) IDMapper` 

Introduce the features to a protein identification file (idXML)- the only way to annotate MS2 spectra for GNPS FBMN  (of later importance)

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_IDMapper.html

In [ ]:
interim= os.path.join("results", "interim")
path= os.path.join(interim, "Requantification")
input_feature_files = sorted(glob.glob(os.path.join(path, "MFD_*.featureXML")))

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

In [ ]:
use_centroid_rt= False
use_centroid_mz= True
protein_ids = []
peptide_ids= []

mapper = IDMapper()

input_mzml_files= sorted(glob.glob(os.path.join(interim, 'Preprocessing', "MapAligned_*.mzML")))

for filename in input_mzml_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    for fmap in feature_maps:
        peptide_ids = []
        protein_ids = []
        if os.path.basename(fmap.getMetaValue("spectra_data")[0].decode())[7:] == os.path.basename(filename)[11:]:
            mapper.annotate(fmap, peptide_ids, protein_ids, use_centroid_rt, use_centroid_mz, exp)
            featureidx_file = os.path.join(path, "IDMapper_" + os.path.basename(filename)[11:-4] +"featureXML")
            FeatureXMLFile().store(featureidx_file, fmap)

#### `7) FeatureGroupingAlgorithmKD `

Feature linker clusters the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (no MS2 data).

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FeatureLinkerUnlabeledKD.html

In [ ]:
interim= os.path.join("results", "interim")
path= os.path.join(interim, "Requantification")

input_feature_files = sorted(glob.glob(os.path.join(path, "IDMapper_*.featureXML")))

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

In [ ]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = os.path.basename(feature_map.getMetaValue("spectra_data")[0].decode())[7:]
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setUniqueIds()
consensus_map.setColumnHeaders(file_descriptions)


Consensus_file= os.path.join(path, "consensus" + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, consensus_map)

# get intensities as a DataFrame
df = consensus_map.get_df()
for cf in consensus_map:
    if cf.metaValueExists("best ion"):
        df["adduct"] = [cf.getMetaValue("best ion") for cf in consensus_map]
        break
df["feature_ids"] = [[handle.getUniqueId() for handle in cf.getFeatureList()] for cf in consensus_map]
df= df.reset_index()
df= df.drop(columns= ["sequence"])
df= df.rename(columns={"RT": "RT(s)", "mz" :"m/z"})
# store as tsv file
df.to_csv(os.path.join("results", "features", "FeatureMatrix_Requantified.tsv"), sep = "\t", index = False)
df